<a href="https://colab.research.google.com/github/EricBartissol/CV/blob/gh-pages/Lectura_historial_generaci%C3%B3n_API_REE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Consulta historial composicion de generacion 

# Import

In [ ]:
import pandas as pd
import requests as requests
import json
import datetime as dt
from dateutil.relativedelta import *

# Configuracion 


fuente:  https://www.ree.es/es/apidatos

Las consultas de composicion de generacion a traves de la API de REE limitan a 744 lecturas horarias (31 dias).

El programa siguiente realiza una serie de consultas para recuperar los datos de consultas entre dos fechas separadas de mas de 31 dias e importarlas en un dataframe.

En el bloque siguiente se definen las fechas de incio y de final de las consulta ambos meses inclusivos.


In [ ]:
inicio = dt.datetime(2014,1,1,0,0)
final = dt.datetime(2016,12,1,0,0)

# Function Request and get data
la fucntion definida a continuacion permite leer los datos del balance electrico de un año completo y guardarlos en un dataframe

In [ ]:
# Entrad un dia del mes 

def generacionRee(fecha):
  """DOCSTRING_DE_FUNCION"""

  # Preparacion datos de consulta
  lang = 'es'
  category = 'balance'
  widget = 'balance-electrico'

  # Preparacion fechas de consulta
  start_date = dt.datetime(fecha.year,fecha.month,1,0,0)
  end_date = start_date+relativedelta(months=1,hours=-1)
  start_date = start_date.strftime('%Y-%m-01T00:00')
  end_date = end_date.strftime('%Y-%m-%dT%H:00')

  # Creacion de la consulta
  query= 'start_date='+start_date+'&end_date='+end_date+'&time_trunc=day'

  # Consulta
  getUrl = 'https://apidatos.ree.es/' + lang + '/datos/' + category + '/' + widget + '?' + query

  # Lectura
  response = requests.get(getUrl).json()

  
  # Preparacion lectura en DataFrame
  df_lectura = pd.DataFrame({'datetime':[],'grupo':[], 'tipo':[], 'agregado':[], 'energia':[]} )
  for i in response['included']:
    grupo=i['type']
    for j in i['attributes']['content']:
      tipo=j['type']
      agregado=j['attributes']['type']
      for k in j['attributes']['values']:
        if agregado == 'distinct':
          datetime=k['datetime']
          energia=k['value']
          df_lectura=df_lectura.append({'datetime':datetime,'grupo':grupo, 'tipo':tipo, 'agregado':agregado, 'energia':energia},ignore_index=True)

  return  df_lectura

# Creacion del DataFrame sobre todo el periodo


In [ ]:
datos = pd.DataFrame({'datetime':[],'grupo':[], 'tipo':[], 'agregado':[], 'energia':[]} )

In [ ]:

fecha = inicio

while fecha<=final:
  datos = datos.append(generacionRee(fecha))
  fecha = fecha+relativedelta(months=1)


# Guardar como csv

In [ ]:
datos.to_csv('generacion.csv',index=False)

In [ ]:
# https://www.ree.es/es/apidatos
# formato:
# > GET /{lang}/datos/{category}/{widget}?[query]
# ejemplo:
# >https://apidatos.ree.es/es/datos/balance/balance-electrico?start_date=2019-01-01T00:00&end_date=2019-01-31T22:00&time_trunc=day


# https://api.esios.ree.es/archives/70/download_json?locale=es&date=2020-01-30